# 1. 요약 : 절차
  
  - 데이터 준비
    - 노랫말(song lyrics) 데이터 받기
    - 데이터 크기: 187088  
      
      

  - 데이터 분석 / 전처리
    - data
      - 문장 길이가 0인 데이터 제외
      - 소문자로 변경
      - 특수문자 및 공백 제외
      - 문장의 시작과 끝을 표시 : <start> + 문장 + <end>  
    - tokenize
      - 구축한 corpus를 tokenize  
    - 데이터 : source, target = 140599, 140599
      - total params = 22,607,961  
      
    
    
  - 학습 : Optimizer를 세가지로 학습
    - Adam optimizer : loss = 1.2485
    - Adadelta optimizer : loss = 1.1361
    - Nadam optimizer : loss = 0.9800
    - 학습한 것을 다시 학습시켜서인지 loss 감소  
      
    
    
  - 학습한 결과를 바탕으로 text 생성
    
result|Adam|Adade|Nadam
---|---|---|---
i love|i love the the for so you you to go|i love the the for into you you|i love the the you i in that had to be for
you love|you love future the so is road that far|you love future the so is road that far|you love future a same that know me around

    
    


# 2. 회고

  - 이번 과제에서도 어김없이 오류가 발생 - 컴에 문제가 있는 듯
    - 주피터 노트북 재설치 해볼 예정  
    
    
  - 다른 과제처럼 그것을 해결하기 위해 노력하기 보다 일단 결과물을 먼저 도출해내자는 생각
    - 구글 코랩에 바로 접속해서 실행시킴.
    - 결과물을 도출하고 여유도 좀 있어서 Optimizer를 바꿔가면서 결과를 지켜봄.
    - Optimizer에 따라 결과가 다르게 나옴.  
    
    
  - 토큰화 하는 부분과 텍스트를 만드는 부분에서 학습이 좀더 필요함.
    - 아직 학습해야할 것이 많음. 
    - 4주차가 되면서 조금 안일한 모습이 보이는 것 같아 다시 초심으로 돌아가자고 다짐.
      - 노드가 밀리지 않도록 열심히 하자고 다짐.
      - 코딩 부분을 좀더 살펴보면서 로직을 이해해야 함. 로직을 모른 채 옮기기만 하는 모습이 비춰짐.
      - 최대한 로직을 학습하고 구글링 하는 노력도 필요.  
        
        
  - 머신러닝을 학습하면서 눈으로만 학습했는데, 아이펠에서는 코딩을 하니 실력이 예전보다 늘어났음을 느낄 수 있음
  

In [ ]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os
import sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/ssac/project/pro_06_lyrics/lyrics_data/*.txt'

In [ ]:
import glob
import pandas as pd

#txt_file_path = os.getenv(data_path)
txt_list = glob.glob(data_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["I'll undress you, 'cause you're tired", 'Cover you as you desire', 'When you fall asleep inside my arms']


In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

I'll undress you, 'cause you're tired
Cover you as you desire
When you fall asleep inside my arms
May not have the fancy things
But I'll give you everything
You could ever want, it's in my arms So baby tell me yes
And I will give you everything
So baby tell me yes
And I will be all yours tonight
So baby tell me yes


In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [ ]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> i ll undress you , cause you re tired <end>',
 '<start> cover you as you desire <end>',
 '<start> when you fall asleep inside my arms <end>',
 '<start> may not have the fancy things <end>',
 '<start> but i ll give you everything <end>',
 '<start> you could ever want , it s in my arms so baby tell me yes <end>',
 '<start> and i will give you everything <end>',
 '<start> so baby tell me yes <end>',
 '<start> and i will be all yours tonight <end>',
 '<start> so baby tell me yes <end>']

In [ ]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.

    
    #jjang : maxlen = 15로 조정
    #tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    5   61 ...    0    0    0]
 [   2 1169    7 ...    0    0    0]
 [   2   46    7 ...    0    0    0]
 ...
 [   2  100    6 ...    0    0    0]
 [   2  122    9 ...    0    0    0]
 [   2   74    6 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f61c37825f8>


In [ ]:
print(tensor[:3, :10])

print(type(tensor), len(tensor))

[[   2    5   61 3121    7    4   67    7   54  617]
 [   2 1169    7   81    7 1255    3    0    0    0]
 [   2   46    7  292 1599  294   13  465    3    0]]
<class 'numpy.ndarray'> 175749


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [ ]:
src_input = tensor[:-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2    5   61 3121    7    4   67    7   54  617    3    0    0    0
    0]
[   2 1169    7   81    7 1255    3    0    0    0    0    0    0    0
    0]


In [ ]:
#jjang : 데이터 분할 

from sklearn.model_selection import train_test_split

enc_train, enc_valid, dec_train, dec_valid= train_test_split(tensor, tensor, test_size=0.2, shuffle=True)

print("Source Train : ", enc_train.shape)
print("Target Train : ", dec_train.shape)


Source Train :  (140599, 15)
Target Train :  (140599, 15)


In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 15, 7001), dtype=float32, numpy=
array([[[ 4.09330642e-05,  1.37340576e-05,  1.61856224e-04, ...,
          1.63501900e-04, -1.60792755e-04,  1.24444836e-04],
        [ 1.81842814e-04,  5.21840004e-04,  4.39726457e-04, ...,
         -1.76459580e-04, -3.33700591e-04,  1.80720526e-04],
        [ 2.93316174e-04,  6.31218660e-04,  4.60614188e-04, ...,
         -4.90534236e-04, -5.03324030e-04,  3.71948758e-04],
        ...,
        [ 6.80436613e-04,  5.24469768e-04,  1.39803218e-03, ...,
          1.09534257e-03, -1.60717114e-03,  8.17703200e-04],
        [ 9.22679203e-04,  5.15878666e-04,  1.72102207e-03, ...,
          1.37571990e-03, -1.74075901e-03,  7.95388594e-04],
        [ 1.20978907e-03,  5.40311972e-04,  2.01766216e-03, ...,
          1.59502355e-03, -1.89046061e-03,  7.71301333e-04]],

       [[ 4.09330642e-05,  1.37340576e-05,  1.61856224e-04, ...,
          1.63501900e-04, -1.60792755e-04,  1.24444836e-04],
        [-3.04567046e-04,  1.07314750e-04,  2.

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# from tensorflow.keras import backend as K
# from tensorflow.keras.layers import *
# from tensorflow.keras.models import *
# from tensorflow.keras.callbacks import * 

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 111s 159ms/step - loss: 1.8783
Epoch 2/10
686/686 [==============================] - 110s 160ms/step - loss: 1.7531
Epoch 3/10
686/686 [==============================] - 110s 160ms/step - loss: 1.6654
Epoch 4/10
686/686 [==============================] - 110s 160ms/step - loss: 1.5945
Epoch 5/10
686/686 [==============================] - 111s 161ms/step - loss: 1.5215
Epoch 6/10
686/686 [==============================] - 110s 161ms/step - loss: 1.4558
Epoch 7/10
686/686 [==============================] - 110s 161ms/step - loss: 1.3940
Epoch 8/10
686/686 [==============================] - 111s 161ms/step - loss: 1.3388
Epoch 9/10
686/686 [==============================] - 110s 161ms/step - loss: 1.2951
Epoch 10/10
686/686 [==============================] - 111s 161ms/step - loss: 1.2485


In [ ]:
def generate_text(model, tokenizer, init_sentence="<start> ", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                    tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(model, tokenizer, init_sentence="<start>i i love")
# <start> i love"는 오류발생
# <start>i love"는 'i'가 누락

'<unk> i love the the for so you you to go <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start>i love")


'<unk> love i just all all back head me head head <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start>you love")
# generate_text(model, tokenizer, init_sentence="<start>i love")와 결과 같음
# <start>you love" : you를 인지하지 못함

'<unk> love i just all all back head me head head <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start>you you love")


'<unk> you love future the so is road that far <end> '

In [ ]:
optimizer = tf.keras.optimizers.Adadelta()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 108s 154ms/step - loss: 1.1693
Epoch 2/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1600
Epoch 3/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1603
Epoch 4/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1521
Epoch 5/10
686/686 [==============================] - 105s 152ms/step - loss: 1.1497
Epoch 6/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1443
Epoch 7/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1454
Epoch 8/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1420
Epoch 9/10
686/686 [==============================] - 105s 152ms/step - loss: 1.1363
Epoch 10/10
686/686 [==============================] - 104s 152ms/step - loss: 1.1361


In [ ]:
generate_text(model, tokenizer, init_sentence="<start>i i love")

'<unk> i love the the for into you you <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start>you you love")


'<unk> you love future the so is road that far <end> '

In [ ]:
optimizer = tf.keras.optimizers.Nadam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 115s 164ms/step - loss: 1.2113
Epoch 2/10
686/686 [==============================] - 112s 164ms/step - loss: 1.1578
Epoch 3/10
686/686 [==============================] - 112s 164ms/step - loss: 1.1256
Epoch 4/10
686/686 [==============================] - 112s 164ms/step - loss: 1.0967
Epoch 5/10
686/686 [==============================] - 112s 164ms/step - loss: 1.0711
Epoch 6/10
686/686 [==============================] - 112s 164ms/step - loss: 1.0485
Epoch 7/10
686/686 [==============================] - 113s 164ms/step - loss: 1.0242
Epoch 8/10
686/686 [==============================] - 112s 164ms/step - loss: 1.0090
Epoch 9/10
686/686 [==============================] - 113s 164ms/step - loss: 0.9917
Epoch 10/10
686/686 [==============================] - 112s 164ms/step - loss: 0.9800


In [ ]:
generate_text(model, tokenizer, init_sentence="<start>i i love")

'<unk> i love the the you i in that had to be for <end> '

In [ ]:
generate_text(model, tokenizer, init_sentence="<start>you you love")

'<unk> you love future a same that know me around <end> '